##### Office Hijinks

# The Lazy Colleague (400 points)

### Backstory

It is very common to work in teams, but it is just as common to find a teammate who decides not to work. However, colleagues do not usually tell the boss, so this individual goes unnoticed. Zenda is supervising four employees, and it is known that one of them never works. But who is it?

### Finding the lazy employee

The project Zenda's team is working on can only be completed if **at least three people** in the team are working. Let's model this situation in a circuit:

![project quantum circuit](../img/project_execution.jpeg)

In this diagram, the qubit $e_i$ refers to the $i$-th employee, which will take the value 1 if this employee is chosen to work on the project. The output state labelled result will take the value 0 if the project was not completed and 1 if it was. Let us imagine that employee $e_1$ is the one who does not work. Then, if we apply the operator to the state $|1\rangle |1\rangle |0\rangle |1\rangle |0\rangle$ (that is, we select $e_1$, $e_2$, and $e_4$ for the project), the output will be $|1\rangle |1\rangle |0\rangle |1\rangle |0\rangle$. As we can see, the last qubit is still $|0\rangle$, i.e. the project has not been carried out. This is because there are only two employees that actually work on the project, and a minimum of three is required.

Zenda wants to know who the lazy worker is, executing as few projects as possible. For this reason, they ask you to help her with your quantum skills. You are asked to discover who the lazy employee is, using a single shot and a single call to the "Project execution" operator.

## Challenge code

On one hand, you are asked to complete `circuit` (you only need to apply gates). You can only call the `project_execution` operator once, which is already incorporated in the template. On the other hand, you must complete `process_output`, which will take the output of your circuit and will return who the lazy guy is.

The `project_execution` function will be generated when testing the solution; if you want to experiment with the function output in the notebook, you can temporarily replace `project_execution` with an operator of the form `qml.MultiControlledX(wires=['e1', 'e2', 'e4', 'result'])`. In this case, the absence of "e3" on the wires means that in this project, "e3" will be the lazy employee. Just remember to switch it back to project_execution before submitting, so that your function uses the correct project_execution during testing!

You may find it useful to do some tests in [Quirk](https://algassert.com/quirk) before you start coding.

### Output

To judge this challenge, we will arbitrarily generate 5000 different projects (`project_execution`), which we will send one by one to the circuit to check that your prediction is correct ("e1", "e2", "e3" or "e4"). Therefore, in this case, there will be no public and private test cases. Good luck!

### Code

In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

In [2]:
dev = qml.device("default.qubit", wires=["e1", "e2", "e3", "e4", "result"], shots=100)
dev.operations.add("op")

wires = ["e1", "e2", "e3", "e4", "result"]

In [3]:
@qml.qnode(dev)
def circuit(project_execution):
    """This is the circuit we will use to detect which is the lazy worker. Remember 
    that we will only execute one shot.

    Args:
        project_execution (qml.ops): 
            The gate in charge of marking in the last qubit if the project has been finished
            as indicated in the statement.

    Returns:
        (numpy.tensor): Measurement output in the 5 qubits after a shot.
    """
    
    # Put your code here #
    
    ## This is Grover search over the reduced 4-dimensional subspace
    # init |0111>|0> + |1011>|0> + |1101>|0> + |1110>|0>
    n = len(wires) - 1
    for i in range(n): # prep i-th qubit
        if i == 0: # non-controlled RX
            qml.RX(np.arcsin(1/np.sqrt(n-i)) * 2, wires=wires[i])
        else: # multi-controlled RX
            qml.ctrl(qml.RX, control=wires[:i])(
                np.arcsin(1/np.sqrt(n-i)) * 2, wires[i]
            )
        qml.PauliX(wires=wires[i]) # rotate i-th qubit to 1

    # phase flip the solution
    # e.g. -|0111>|-> + |1011>|-> + |1101>|-> + |1110>|->
    qml.PauliX(wires="result")
    qml.Hadamard(wires="result")
    project_execution(wires=wires)
    
    # reverse the initialisation
    for i in range(n)[::-1]: # prep i-th qubit
        qml.PauliX(wires=wires[i])
        if i == 0: # non-controlled RX
            qml.RX(-np.arcsin(1/np.sqrt(n-i)) * 2, wires=wires[i])
        else: # multi-controlled RX
            qml.ctrl(qml.RX, control=wires[:i])(
                -np.arcsin(1/np.sqrt(n-i)) * 2, wires[i]
            )
    
    # phase flip all the initial state |0000>
    for i in wires[:-1]:
        qml.PauliX(wires=i)
    qml.ctrl(qml.PauliZ, control=wires[:-2])(wires=wires[-2])
    for i in wires[:-1]:
        qml.PauliX(wires=i)
        
    # redo the initialisation
    for i in range(n): # prep i-th qubit
        if i == 0: # non-controlled RX
            qml.RX(np.arcsin(1/np.sqrt(n-i)) * 2, wires=wires[i])
        else: # multi-controlled RX
            qml.ctrl(qml.RX, control=wires[:i])(
                np.arcsin(1/np.sqrt(n-i)) * 2, wires[i]
            )
        qml.PauliX(wires=wires[i])
    
    return qml.sample(wires=dev.wires[:-1])

def process_output(measurement):
    """This function will take the circuit measurement and process it to determine who is the lazy worker.

    Args:
        measurement (numpy.tensor): Measurement output in the 5 qubits after a shot.

    Returns:
        (str): This function must return "e1", "e2", "e3", or "e4" - the lazy worker.
    """
    if np.all(measurement == [0,1,1,1]):
        return "e1"
    elif np.all(measurement == [1,0,1,1]):
        return "e2"
    elif np.all(measurement == [1,1,0,1]):
        return "e3"
    else:
        return "e4"

In [4]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    return None

def check(solution_output: str, expected_output: str) -> None:
    samples = 5000

    solutions = []
    output = []

    for s in range(samples):
        lazy = np.random.randint(0, 4)
        no_lazy = list(range(4))
        no_lazy.pop(lazy)

        def project_execution(wires):
            class op(qml.operation.Operator):
                num_wires = 5

                def compute_decomposition(self, wires):
                    raise ValueError("You cant descompose this gate")

                def matrix(self):
                    m = np.zeros([32, 32])
                    for i in range(32):
                        b = [int(j) for j in bin(64 + i)[-5:]]
                        if sum(np.array(b)[no_lazy]) == 3:
                            if b[-1] == 0:
                                m[i, i + 1] = 1
                            else:
                                m[i, i - 1] = 1
                        else:
                            m[i, i] = 1
                    return m

            op(wires=wires)
            return None

        out = circuit(project_execution)
        solutions.append(lazy + 1)
        output.append(int(process_output(out)[-1]))

    assert np.allclose(
        output, solutions, rtol=1e-4
    ), "Your circuit does not give the correct output."

    ops = [op.name for op in circuit.tape.operations]
    assert ops.count("op") == 1, "You have used the oracle more than one time."

In [5]:
test_cases = [['No input', 'No output']]

In [6]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input 'No input'...
Correct!
